## CNN Text Classification with Keras and Theano

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import _pickle as cPickle # python3 shit
import re, sys, os
os.environ['KERAS_BACKEND']='theano'
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import chain
from nltk.classify import NaiveBayesClassifier, accuracy
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model
print("Using Keras version",keras.__version__)

Read from dataset (`csv` file)

In [ ]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Functions to pre-process data to force lower-case, remove special characters and [stop words](https://en.wikipedia.org/wiki/Stop_words); and to vectorise the labelling.

In [ ]:
stop_words = list(get_stop_words('en'))         #About 900 stop words
nltk_words = list(stopwords.words('english'))   #About 150 stop words
stop_words.extend(nltk_words)

def clean_tweet( tweet):
        tweet = tweet.lower() 
        intem = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        filtered_tweet = ""
        for word in intem:
            if word not in stopwords.words("english"):
                filtered_tweet = filtered_tweet + " " + word
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 1
    elif label == "sadness":return 2
    elif label == "enthusiasm":return 3
    elif label == "neutral":return 4
    elif label == "worry":return 5
    elif label == "surprise":return 6
    elif label == "love":return 7
    elif label == "fun":return 8
    elif label == "hate":return 9
    elif label == "happiness":return 10
    elif label == "boredom":return 11
    elif label == "relief":return 12
    elif label == "anger":return 13
    else:return 0

In [ ]:
texts, labels = [], []
print("Cleaning and generating text and label Lists...")
total = data_train.content.shape[0]
for i in range(data_train.content.shape[0]):
    print("Progress: ",round(i/total*100,2),"   ",end="\r")
    texts.append(clean_tweet(data_train.content[i]))
    sys.stdout.write("\033[F")
    labels.append(vectorise_label(data_train.sentiment[i]))
print("Progress: ",100,"\nComplete!")

In [ ]:
MAX_NB_WORDS = 40000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [ ]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of entries in each category:')
print("Training:\n",y_train.sum(axis=0))
print("Validation:\n",y_val.sum(axis=0))

### Preparing the Embedding layer
Compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings: [GloVe](https://nlp.stanford.edu/projects/glove/) vectors from Stanford NLP. For new words, a "randomised vector" will be created.

In [ ]:
EMBEDDING_DIM = 200
GLOVE_DIR = "dataset/glove/glove.twitter.27B.200d.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
print("Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        print("Error:",e)
f.close()
print("Done.\nProceeding with Embedding Matrix...")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("Completed!")

After computing our embedding matrix, load this embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

### Simplified 1D CNN
Structure: total 128 filters with size 5 and max pooling of 5 and 35. [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool3 = MaxPooling1D(5)(l_cov3)
l_cov4 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool4 = MaxPooling1D(35)(l_cov4)  # global max pooling
l_flat = Flatten()(l_pool4)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(14, activation='softmax')(l_dense) # 14 categories

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
print("Model Summary:")
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=128)

Deeper Convolutional neural network:In [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [ ]:
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=20, batch_size=50)

### TODO

 - tune hyper-parameters
 - improve preprocessing
 - use drop-out